In [26]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
data = load_iris()
iris = pd.DataFrame(data.data, columns=data.feature_names)
iris['class'] = pd.Series(data.target)
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [27]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [28]:
renames = {name: name.replace(' (cm)', '').replace(' ', '_') for name in iris.columns}
renames['class'] = 'cls'
iris = iris.rename(columns=renames)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,cls
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [29]:
petal_length = ctrl.Antecedent(np.linspace(1.0, 7.0, 11), 'petal_length')
petal_width = ctrl.Antecedent(np.linspace(0.0, 2.5, 11), 'petal_width')
petal_length.automf(names="short medium long".split())
petal_width.automf(names="narrow medium wide".split())

In [30]:
petal_width.view()

In [31]:
# Creating a consequent variable for each iris type, with a single membership term 'likely'

setosa = ctrl.Consequent(np.linspace(0, 1, 10), 'setosa', "som")
setosa['likely'] = fuzz.trimf(setosa.universe, (0., 1., 1.))

versicolor = ctrl.Consequent(np.linspace(0, 1, 10), 'versicolor', "som")
versicolor['likely'] = fuzz.trimf(versicolor.universe, (0., 1., 1.))

verginica = ctrl.Consequent(np.linspace(0, 1, 10), 'verginica', "som")
verginica['likely'] = fuzz.trimf(verginica.universe, (0., 1., 1.))


In [32]:
# create rules

rule1 = ctrl.Rule(petal_width['narrow'] & petal_length['short'], setosa['likely'])
rule2 = ctrl.Rule(petal_width['medium'] & petal_length['medium'], versicolor['likely'])
rule3 = ctrl.Rule(petal_width['wide'] & petal_length['long'], verginica['likely'])

In [33]:
iris_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
iris_classifier = ctrl.ControlSystemSimulation(iris_ctrl)

In [34]:
%%time
predicted = []
for (plength, pwidth) in zip (iris.petal_length, iris.petal_width):
    iris_classifier.inputs({'petal_length': plength, 'petal_width': pwidth})
    iris_classifier.compute()
    predicted.append(np.argmax([iris_classifier.output['setosa'], iris_classifier.output['versicolor'], iris_classifier.output['verginica']]))

CPU times: user 99 ms, sys: 8 ms, total: 107 ms
Wall time: 101 ms


In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(iris.cls, predicted)

array([[50,  0,  0],
       [ 0, 50,  0],
       [ 0, 20, 30]])

In [38]:
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
height = ctrl.Antecedent(np.linspace(100, 250, 100), "height (cm)")
height.automf(names="tiny short average tall giant".split())
height.view()


In [9]:
height.view?

Signature: height.view(*args, **kwargs)
Docstring: <no docstring>
File:      ~/anaconda3/envs/eff/lib/python3.9/site-packages/skfuzzy/control/fuzzyvariable.py
Type:      method


In [35]:
rule1.view()

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:>)

In [37]:
setosa.view()